In [1]:
import pandas as pd
from glob import glob
import os
import numpy as np

In [2]:
files = glob('C:\\Users\\jlandman\\Desktop\\newData\\CL_Schaefer\\GlaThiDa2-0_MS*_Johannes.xls')
print(files)

['C:\\Users\\jlandman\\Desktop\\newData\\CL_Schaefer\\GlaThiDa2-0_MS2_Johannes.xls', 'C:\\Users\\jlandman\\Desktop\\newData\\CL_Schaefer\\GlaThiDa2-0_MS_Johannes.xls']


In [3]:
thin_dict = {'Del Potro':8,
             'Tronquitos':6,
             'Tapado':12,
             'Juncal Norte': 20,   
             'Rio Blanco':24,
             'Tupungatito':22,
             'Marmolejo':26    
}

In [32]:
data_new = pd.DataFrame([])

for file in files:
    fname = os.path.basename(file)
    data = pd.read_excel(file, sheetname=1)
    
    for glacier in np.unique(data.GLACIER_NAME.values):
        temp = data[data.GLACIER_NAME==glacier]
        
        gtd_id = temp[temp.GLACIER_NAME==glacier].GlaThiDa_ID.values[0]
        pol_u = temp[temp.GLACIER_NAME==glacier].POLITICAL_UNIT.values[0]
        surv_date = temp[temp.GLACIER_NAME==glacier].SURVEY_DATE.values[0]
        remarks = temp[temp.GLACIER_NAME==glacier].REMARKS.iloc[0]
        gname = glacier
        
        temp = temp.groupby(temp.index // thin_dict[glacier]).mean() # average over every x values as specified above
        
        temp['POLITICAL_UNIT'] = pol_u
        temp['GLACIER_NAME'] = gname
        temp['REMARKS'] = remarks
        temp['THICKNESS_UNCERTAINTY'] = np.nan
        temp['DATA_FLAG'] = np.nan
        temp['GlaThiDa_ID'] = gtd_id
        temp['SURVEY_DATE'] = surv_date
        temp['POINT_ID'] = range(1,len(temp)+1)
        
        try:
            temp['ELEVATION'] = temp['ELEVATION'].map('{:.0f}'.format)
        except KeyError:
            temp['ELEVATION'] = np.nan
        
            
        data_new = data_new.append(temp, ignore_index=True)

    
data_new['THICKNESS'] = data_new['THICKNESS'].map('{:.1f}'.format)
data_new = data_new[['GlaThiDa_ID','POLITICAL_UNIT', 'GLACIER_NAME', 'SURVEY_DATE', 'POINT_ID',  'POINT_LAT',  'POINT_LON',  'ELEVATION', 
                     'THICKNESS', 'THICKNESS_UNCERTAINTY', 'DATA_FLAG', 'REMARKS']]

data_new = data_new.sort_values(by=['GlaThiDa_ID','POINT_ID'])
data_new.to_excel(os.path.join(os.path.dirname(file),fname.split('.')[0]+'_thinned.xlsx'), index=False)

C:\Users\jlandman\Anaconda3\lib\site-packages\ipykernel\__main__.py:40: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
